## Instalamos Paquetes y cargamos librerias

In [181]:
#Limpiamos memoria
rm(list=ls());
is.installed <- function(mypkg) {
  is.element(mypkg, installed.packages()[,1]) 
}

#Instalamos paquetes para agrupar csv
if (!is.installed("data.table")){
  install.packages("data.table",repos = "http://cran.us.r-project.org")
};
#Intslamos paquete para manipular ts
if (!is.installed("xts")){
  install.packages("xts",repos = "http://cran.us.r-project.org")
};
#Instalamos paquete para agrupar datos
if (!is.installed("dplyr")){
  install.packages("dplyr",repos = "http://cran.us.r-project.org")
};

# Instalamos paquete para leer exccels
if (!is.installed("readxl")){
  install.packages("readxl",repos = "http://cran.us.r-project.org")
};


In [182]:
# Cargamos las librerias
library(data.table)
library(xts)
library(dplyr)    # loads %>%
library(readxl)

## Cargamos datos calidad del aire 2001-2018 Madrid

In [ ]:
#Cargamos los datos de los anos en un dataset (utilziamos read.csv ya que el separador es,)

data2001<- read.csv(file = "data/madrid_2001.csv",header = TRUE);
data2002<- read.csv(file = "data/madrid_2002.csv",header = TRUE);
data2003<- read.csv(file = "data/madrid_2003.csv",header = TRUE);
data2004<- read.csv(file = "data/madrid_2004.csv",header = TRUE);
data2005<- read.csv(file = "data/madrid_2005.csv",header = TRUE);
data2006<- read.csv(file = "data/madrid_2006.csv",header = TRUE);
data2007<- read.csv(file = "data/madrid_2007.csv",header = TRUE);
data2008<- read.csv(file = "data/madrid_2008.csv",header = TRUE);
data2009<- read.csv(file = "data/madrid_2009.csv",header = TRUE);
data2010<- read.csv(file = "data/madrid_2010.csv",header = TRUE);
data2011<- read.csv(file = "data/madrid_2011.csv",header = TRUE);
data2012<- read.csv(file = "data/madrid_2012.csv",header = TRUE);
data2013<- read.csv(file = "data/madrid_2013.csv",header = TRUE);
data2014<- read.csv(file = "data/madrid_2014.csv",header = TRUE);
data2015<- read.csv(file = "data/madrid_2015.csv",header = TRUE);
data2016<- read.csv(file = "data/madrid_2016.csv",header = TRUE);
data2017<- read.csv(file = "data/madrid_2017.csv",header = TRUE);
data2018<- read.csv(file = "data/madrid_2018.csv",header = TRUE);

In [ ]:
# unimos los datos 
data <- data.table::rbindlist(list(data2001,data2002,data2003,data2004,data2005,data2006,
                                   data2007,data2008,data2009,data2010,data2011,data2012,
                                   data2013,data2014,data2015,data2016,data2017,data2018),
                                   fill = TRUE);

# convertimos la lista en un dataset
datasetMadrid <- as.data.frame(data); 

# liberamos memoria
rm(data)

## Visualizacion inicial datos calidad del aire

In [ ]:
# observamos que todos los datos se han unido, aplicando head y tail al dataset 
# para ver cuando empiezan y acaban fechas

head(datasetMadrid)

In [ ]:
tail(datasetMadrid)

## Carga datos Precipitaciones 2001-2018

In [ ]:
#Cargamos los datos de las precipitaciones
rainfall_2001<- readxl::read_xlsx("data/rainfall_2001.xlsx",col_names = TRUE);
rainfall_2002<- readxl::read_xlsx("data/rainfall_2002.xlsx",col_names = TRUE);
rainfall_2003<- readxl::read_xlsx("data/rainfall_2003.xlsx",col_names = TRUE);
rainfall_2004<- readxl::read_xlsx("data/rainfall_2004.xlsx",col_names = TRUE);
rainfall_2005<- readxl::read_xlsx("data/rainfall_2005.xlsx",col_names = TRUE);
rainfall_2006<- readxl::read_xlsx("data/rainfall_2006.xlsx",col_names = TRUE);
rainfall_2007<- readxl::read_xlsx("data/rainfall_2007.xlsx",col_names = TRUE);
rainfall_2008<- readxl::read_xlsx("data/rainfall_2008.xlsx",col_names = TRUE);
rainfall_2009<- readxl::read_xlsx("data/rainfall_2009.xlsx",col_names = TRUE);
rainfall_2010<- readxl::read_xlsx("data/rainfall_2010.xlsx",col_names = TRUE);
rainfall_2011<- readxl::read_xlsx("data/rainfall_2011.xlsx",col_names = TRUE);
rainfall_2012<- readxl::read_xlsx("data/rainfall_2012.xlsx",col_names = TRUE);
rainfall_2013<- readxl::read_xlsx("data/rainfall_2013.xlsx",col_names = TRUE);
rainfall_2014<- readxl::read_xlsx("data/rainfall_2014.xlsx",col_names = TRUE);
rainfall_2015<- readxl::read_xlsx("data/rainfall_2015.xlsx",col_names = TRUE);
rainfall_2016<- readxl::read_xlsx("data/rainfall_2016.xlsx",col_names = TRUE);
rainfall_2017<- readxl::read_xlsx("data/rainfall_2017.xlsx",col_names = TRUE);
rainfall_2018<- readxl::read_xlsx("data/rainfall_2018.xlsx",col_names = TRUE);

In [ ]:
# unimos los datos
rainfall <- data.table::rbindlist(list(rainfall_2001,rainfall_2002,rainfall_2003,rainfall_2004,
              rainfall_2005,rainfall_2006,rainfall_2007,rainfall_2008,rainfall_2009,rainfall_2010,
              rainfall_2011,rainfall_2012,rainfall_2013,rainfall_2014,rainfall_2015,rainfall_2016,
rainfall_2017,rainfall_2018), fill = TRUE)

# convertimos la lista en un dataset
datasetRainfall <- as.data.frame(rainfall);

# liberamos memoria
rm(rainfall)

# cambiamos nombre de las columnas
data.table::setnames(datasetRainfall, "FECHA", "date")
data.table::setnames(datasetRainfall, "Precipitación: l/m2", "l/m2")

## Visualizacion inicial datos precipitaciones

In [ ]:
##observamos que todos los datos se han unido, aplicando head y tail al dataset 
# para ver cuando empiezan y acaban fechas
head(datasetRainfall)

In [ ]:
tail(datasetRainfall)

## Limpieza de datos

Primero debemos tranformar los NaN en NA para obtener los datos que puedens ser utilizados

In [ ]:
is.nan.data.frame <- function(x)
    do.call(cbind,lapply(x,is.nan))
datasetMadrid <- as.data.frame(datasetMadrid)
datasetMadrid[is.nan(datasetMadrid)] <- NA
tail(datasetMadrid)

In [ ]:
#Limpieza de datos datasetMadrid

#1) Calculamos el porcenje de NA para descartar aquellas columnas cuyo porcentage sea muy alto
naPerColumn <- colSums(is.na(datasetMadrid))

#2) Todos los gases tienen al menos un NA por , tanto, estudiamos gases a descartar, para ello
#quitamos los demás parámetros
percentageNA <- round(naPerColumn/nrow(datasetMadrid) *100, digits = 2)
percentageNA <- percentageNA[percentageNA > 0]
percentageNA <- percentageNA[order(percentageNA)]
percentageNA

In [ ]:
#2)Pintamos grafica de barras para observar volumen NA en los gases contaminantes
barplot(percentageNA, main ="Porcentaje de NA",xlab = "Gas contaminante",ylab = "%",
        ylim = c(0,100), las=3, cex.names=.5, col = gray.colors(17))

In [ ]:
#Nos quedamos con aquellos gases que tengan menos del 35% de NA
percentageNA[percentageNA < 35]

In [ ]:
#Procedemos a quitar el resto de columnas
datasetMadrid$BEN = NULL;
datasetMadrid$EBE = NULL;
datasetMadrid$MXY = NULL;
datasetMadrid$NMHC = NULL;
datasetMadrid$NOx = NULL;
datasetMadrid$OXY = NULL;
datasetMadrid$PXY = NULL;
datasetMadrid$TCH = NULL;
datasetMadrid$TOL = NULL;
datasetMadrid$PM25 = NULL;
datasetMadrid$NO = NULL;
datasetMadrid$CH4 = NULL;

#Mostramos dataframe para observar cambios
head(datasetMadrid)
tail(datasetMadrid)

In [ ]:
# Liberamos memoria
rm(percentageNA)
rm(naPerColumn)

## Agrupacion de datos

In [ ]:
##Se ordenan los datos por estacion, después se comprimen todas las horas 
##de un dia para cada estacion en un solo dia

datasetMadrid<- datasetMadrid[order(datasetMadrid$station),]
head(datasetMadrid, n=24)
tail(datasetMadrid)

### Cambios de formato en la fecha

In [ ]:
datasetMadrid$date <- as.Date(datasetMadrid$date, format = "%Y-%m-%d")

In [ ]:
head(datasetMadrid, n = 24)
tail(datasetMadrid)

In [ ]:
madrid <- datasetMadrid %>%
  group_by(station,date) %>%
  summarise(CO = mean(CO, na.rm = TRUE),
            NO_2 = mean(NO_2, na.rm = TRUE),
            O_3 = mean(O_3, na.rm = TRUE),
            PM10 = mean(PM10, na.rm = TRUE),
            SO_2 = mean(SO_2, na.rm = TRUE))
head(madrid)
tail(madrid)
rm(datasetMadrid)

#### Observamos que la estacion `28079099` no tiene muestras desde 2011, se debe a que fue dada de baja ese año por lo que se procede a eliminar del estudio. A continuación mostramos el final del data.frame para observar que ya no se encuentra dicha estación.

In [178]:
madrid <-  madrid[!madrid$station == 28079099,]
tail(madrid)

station,date,CO,NO_2,O_3,PM10,SO_2
28079060,2018-04-26,NaN,29.041667,77.25000,42.54167,NaN
28079060,2018-04-27,NaN,20.958333,81.83333,29.57143,NaN
28079060,2018-04-28,NaN,8.375000,87.20833,10.33333,NaN
28079060,2018-04-29,NaN,6.958333,93.08333,14.45833,NaN
28079060,2018-04-30,NaN,13.708333,83.33333,12.91667,NaN
28079060,2018-05-01,NaN,14.000000,81.00000,6.00000,NaN


In [179]:
datasetRainfall$date <- as.Date(datasetRainfall$date)
rainfall<-datasetRainfall
rm(datasetRainfall)
head(rainfall)

date,l/m2
2001-01-01,4.3
2001-01-02,0.1
2001-01-03,2.2
2001-01-04,2.4
2001-01-05,13.3
2001-01-06,0.0


In [180]:
#Cruzamos los datos de las precipitaciones y la contaminacion por fecha 
data <- merge ( x = madrid, y = rainfall, by = "date")
head(data[order(data$station),])

,date,station,CO,NO_2,O_3,PM10,SO_2,l/m2
32,2001-01-02,28079001,1.243333,84.53750,12.576667,36.92455,20.28083,0.1
57,2001-01-03,28079001,1.242917,79.75125,10.961250,37.21292,16.23917,2.2
80,2001-01-04,28079001,1.134167,75.39250,12.002917,35.03667,16.75125,2.4
116,2001-01-05,28079001,1.535833,77.14083,8.755833,34.27625,16.00958,13.3
138,2001-01-06,28079001,1.009167,64.47708,13.955417,17.03792,15.35417,0.0
168,2001-01-07,28079001,1.016250,67.06125,12.776250,20.45500,18.43042,0.0
